In [1]:
import org.apache.spark

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-T2P39KLE:4040
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1739306401114)
SparkSession available as 'spark'


import org.apache.spark


In [2]:
val path_to_bigDatasets = "../../../../datasets/big/"
val path_to_datasets = "../../../../datasets/"

val path_Fullml_posts = path_to_bigDatasets + "the-reddit-covid-dataset-posts.csv"
val path_Fullml_comments = path_to_bigDatasets + "the-reddit-covid-dataset-comments.csv"

val path_sample_posts = path_to_datasets + "postsSample.csv"
val path_sample_comments = path_to_datasets + "commentsSample.csv"

path_to_bigDatasets: String = ../../../../datasets/big/
path_to_datasets: String = ../../../../datasets/
path_Fullml_posts: String = ../../../../datasets/big/the-reddit-covid-dataset-posts.csv
path_Fullml_comments: String = ../../../../datasets/big/the-reddit-covid-dataset-comments.csv
path_sample_posts: String = ../../../../datasets/postsSample.csv
path_sample_comments: String = ../../../../datasets/commentsSample.csv


In [3]:
import java.util.Calendar
import org.apache.spark.sql.SaveMode
import org.apache.spark.HashPartitioner

object CovidConversationsParser {

    val commaSplit = ","
    val quotes = "\""

    /** Convert from timestamp (String) to year (Int) */
    def yearFromTimestamp(timestamp: String): Int = {
        val cal = Calendar.getInstance()
        cal.setTimeInMillis(timestamp.trim.toLong * 1000L)
        cal.get(Calendar.YEAR)
    }

    /** Function to parse reddit posts
    *
    *  @param line line that has to be parsed
    *  @return tuple containing id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score. none in case of input errors
    */
    def parseRedditPost(line: String): Option[(String, String, String, Boolean, Int, String, String, String, String, String, Int)] = {
        try {
            val input = line.split(commaSplit)
            if(input.size != 12) {
                return Some(("", "", "", false, -1, "", "", "", "", "", -1))
            }
            var url = "None"
            if(input(8).trim.nonEmpty) {
                url = input(8).trim
            }
            var selftext = "None"
            if(input(9).trim.nonEmpty) {
                selftext = input(9).trim
            }
            val number = input(11).trim.replaceAll(quotes, "")
            Some((input(1).trim, input(2).trim, input(3).trim, input(4).trim.toBoolean, yearFromTimestamp(input(5)), input(6).trim, input(7).trim, url, selftext, input(10).trim , number.toInt))
        } catch {
            case _: Exception => None
        }
    }

    /** Function to parse reddit comments
    *
    *  @param line line that has to be parsed
    *  @return tuple containing id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score. none in case of input errors
    */
    def parseRedditComment(line: String): Option[(String, String, String, Boolean, Int, String, String, Double, Int)] = {
        try {
            val input = line.split(commaSplit)
            if(input.size != 10) {
                return Some(("", "", "", false, -1, "", "", -0.1, -1))
            }
            val number = input(9).trim.replaceAll(quotes, "")
            Some((input(1).trim, input(2).trim, input(3).trim, input(4).trim.toBoolean,yearFromTimestamp(input(5)), input(6).trim, input(7).trim, input(8).trim.toDouble, number.toInt))
        } catch {
            case _: Exception => None
        }
    }
}

import java.util.Calendar
import org.apache.spark.sql.SaveMode
import org.apache.spark.HashPartitioner
defined object CovidConversationsParser


In [ ]:
val array = Array("","10200SB1169ham002- 2 -LRB102 04980 LNS 30154 a1",
                  "post,qftn1l,4ir4l2,deplatformed_,false,1635205611,https://old.reddit.com/r/Deplatformed_/comments/qftn1l/how_many_lives_have_been_lost_due_to_covid19/,freeworldnews.tv,https://freeworldnews.tv/watch?id=617610dbb48cdd1309da36bb,,How many lives have been lost due to Covid-19 'treatment hesitancy' from the left?,1",
                  "post,qftm4e,2qh1i,askreddit,false,1635205525,https://old.reddit.com/r/AskReddit/comments/qftm4e/does_anyone_think_this_covid_drama_will_go_away/,self.askreddit,,[removed],Does anyone think this covid drama will go away? Or will it stick around now like the common cold?,1", "- Mission Memorial Hospital, Chilliwack General Hospital (Fraser Health);",
                  "post,qfsoln,2qs1t,newyorkcity,false,1635202606,https://old.reddit.com/r/newyorkcity/comments/qfsoln/live_now_78pm_pix11_holds_their_2021_nyc_mayoral/,pix11.com,https://pix11.com/news/politics/new-york-elections/pix-on-politics-adams-sliwa-to-face-off-in-nyc-mayoral-forum-monday/,,\"Live Now (7-8PM): Pix11 holds their 2021 NYC mayoral forum, with frontrunner candidates Eric Adams (D), and Curtis Sliwa (R) answering questions on city issues such as COVID-19, vaccine mandates, schools, crime, youth violence and much more. (10/25/2021)\",1")
val testRDDPost = sc.parallelize(array, 2)
val testParser = testRDDPost.flatMap(CovidConversationsParser.parseRedditPost)
    .filter(x => x != ("", "", "", false, -1, "", "", "", "", "", -1))
print(testParser.first())

In [ ]:
val commentsArray = Array("They were very specific about what they were and weren't studying.  The article you linked from  Nature was less specific.  Your summary is completely mischaracterizing the study you're trying to use as evidence for your position.\",0.8976,1", "======================", "comment,hi1vjyo,2y77d,antiwork,false,1635206288,https://old.reddit.com/r/antiwork/comments/qft1w1/i_work_for_a_lab_doing_covid_testing_everyday_ive/hi1vjyo/,Can I come love with you? I can do Covid testing for free and take up very little space.,0.8176,11"
        ,"If there is an issue with this post and it breaks the rules, please use the report button AND message the Moderators.","comment,hi1uwl8,2ra72,nursing,false,1635205980,https://old.reddit.com/r/nursing/comments/qfkl6h/i_wish_obituaries_for_unvaccinated_that_die_of/hi1uwl8/,\"Yeah, apparently the cancer is what did Colin Powell...and not, you know, the Covid breakthrough from his immune system being absolutely shot.\",-0.25,18", "comment,hi1uj9b,3fmhe,enough_sanders_spam,false,1635205810,https://old.reddit.com/r/Enough_Sanders_Spam/comments/qfb1vm/mondays_general_discussion_roundtable_10252021/hi1uj9b/,\"&gt;[The most hate crimes in the U.S. since 2001 were reported last year, according to newly released data from the FBI that also revealed a stunning 76% increase in anti-Asian hate crimes.](https://www.huffpost.com/entry/fbi-2020-hate-crimes-asian-americans_n_6176fd81e4b010d933148e55)", "comment,hi1tnwd,2reak,marriage,false,1635205404,https://old.reddit.com/r/Marriage/comments/qft3pf/should_i_feel_guilty_for_hanging_out_with_friends/hi1tnwd/,Hahaha well that is a perspective to look at and understand. I hope his insecurities aren’t as full throttle as yours. Unfortunately fuckfests aren’t possible due to Covid restrictions in my state right now.,0.5267,3")
val testRDDComment = sc.parallelize(commentsArray, 2)
val testCommentParser = testRDDComment.flatMap(CovidConversationsParser.parseRedditComment)
                .filter(x => x != ("", "", "", false, -1, "", "", -0.1, -1))
println(testCommentParser.first())

In [ ]:
val sampleFile = path_to_datasets + "sampleTest.csv"
val sampleFileRDD = sc.textFile(sampleFile)
val parserTest = sampleFileRDD.flatMap(CovidConversationsParser.parseRedditPost)
                 .filter(x => x != ("", "", "", false, -1, "", "", "", "", "", -1))
//println(sampleFileRDD.first().getClass.getSimpleName)
println(parserTest.count())

In [ ]:
val sampleCommentFile = path_to_datasets + "sampleCommentTest.csv"
val sampleCommentRDD = sc.textFile(sampleCommentFile)
val parserCommentTest = sampleCommentRDD .flatMap(CovidConversationsParser.parseRedditComment)
                 .filter(x => x != ("", "", "", false, -1, "", "", -0.1, -1))
//println(sampleFileRDD.first().getClass.getSimpleName)
println(parserCommentTest.count())

In [ ]:
//per recuperare dei samples dei files
//sc.textFile(path_Fullml_posts).sample(false, 0.05).coalesce(1).toDF().write.format("csv").mode(SaveMode.Overwrite).save("../../../../datasets/sample")
sc.textFile(path_Fullml_comments).sample(false, 0.02).coalesce(1).toDF().write.format("csv").mode(SaveMode.Overwrite).save("../../../../datasets/sample")

In [4]:
/*getting RDDs of Comments and Posts*/
val rddPosts = sc.textFile(path_sample_posts).flatMap(CovidConversationsParser.parseRedditPost).filter(x => x != ("","","",false,-1,"","","","","",-1))
val rddComments = sc.textFile(path_sample_comments).flatMap(CovidConversationsParser.parseRedditComment)
.filter(x => x != ("", "", "", false, -1, "", "", -0.1, -1))
rddComments.first()

rddPosts: org.apache.spark.rdd.RDD[(String, String, String, Boolean, Int, String, String, String, String, String, Int)] = MapPartitionsRDD[3] at filter at <console>:32
rddComments: org.apache.spark.rdd.RDD[(String, String, String, Boolean, Int, String, String, Double, Int)] = MapPartitionsRDD[7] at filter at <console>:34
res0: (String, String, String, Boolean, Int, String, String, Double, Int) = (hi1vjyo,2y77d,antiwork,false,2021,https://old.reddit.com/r/antiwork/comments/qft1w1/i_work_for_a_lab_doing_covid_testing_everyday_ive/hi1vjyo/,Can I come love with you? I can do Covid testing for free and take up very little space.,0.8176,11)


In [5]:
//PART 1: Aggregate on temporal dimension and obtain percentage of posts classified as NSFW
//Posts are :(id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score)
//(x._1, x._3, x._4)
val percentageNSFWPosts = rddPosts.map(x => (x._5, x._4))  // (created_utc, nsfw flag)
.groupByKey()
.mapValues({case (nsfwCount) =>
    val totalPostsAtTime = nsfwCount.size
    val totNSFWPost = nsfwCount.count(el => el == true)
    ((totNSFWPost * 100) / totalPostsAtTime).toDouble
})
//.coalesce(1) //riduzione a singola partizione (per analisi, con singola partizione abbiamo singolo file da analizzare)
//.collect()

percentageNSFWPosts: org.apache.spark.rdd.RDD[(Int, Double)] = MapPartitionsRDD[10] at mapValues at <console>:33


In [ ]:
val percentageNSFWPosts = rddPosts.map(x => (x._5, (x._1, x._3, if (x._4) 1 else 0))) // (timestamp, (id, subreddit_name, nsfw flag))
    .aggregateByKey((0, 0))( // (nsfwCount, totalCount)
    // Accumulator for each partition
    (acc, value) => (acc._1 + value._3, acc._2 + 1),
    // Combiner for merging partitions
    (acc1, acc2) => (acc1._1 + acc2._1, acc1._2 + acc2._2)
).map { case (timestamp, (nsfwCount, totalCount)) =>
    val percentage = (nsfwCount.toDouble / totalCount) * 100
    (timestamp, rddPosts.filter(_._5 == timestamp).first()._1,  // id
    rddPosts.filter(_._5 == timestamp).first()._3,  // subreddit_name
    percentage)
}.coalesce(1) //riduzione a singola partizione (per analisi, con singola partizione abbiamo singolo file da analizzare)
.collect()

In [6]:
//PART 2: Aggregate on temporal dimension and obtain average sentiment in comments and percentage of comments classified as NSFW
//Comments are: (id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score)
val avgSentimentWithNSFWPost = rddComments.map(x => (x._5, (x._4, x._8)))
.groupByKey()
.mapValues({case (nsfwAndSentimentCount) =>
    val totalPostsAtTime = nsfwAndSentimentCount.size
    val totNSFWPost = nsfwAndSentimentCount.count(el => el._1 == true)
    val percentage = ((totNSFWPost * 100) / totalPostsAtTime).toDouble
    val totSentiment = nsfwAndSentimentCount.map(x => x._2).foldLeft(0.0)(_ + _)
    (percentage, totSentiment / totalPostsAtTime)
})
//.coalesce(1) //riduzione a singola partizione (per analisi, con singola partizione abbiamo singolo file da analizzare)
//.collect()

avgSentimentWithNSFWPost: org.apache.spark.rdd.RDD[(Int, (Double, Double))] = MapPartitionsRDD[13] at mapValues at <console>:32


In [ ]:
/*.mapValues({case (nsfwCount, sentimentCount) =>
val totalPostsAtTime = nsfwCount.size
val totNSFWPost = nsfwCount.count(el => el == true)
((totNSFWPost * 100) / totalPostsAtTime).toDouble
})*/
/*
val avgSentimentWithNSFWPost = rddComments.map(x => (x._5, (if (x._4) 1 else 0, x._8)))  // (created_utc, (nsfw flag, sentiment))
.reduceByKey((a, b) => (a._1 + b._1, a._2 + b._2))  // aggregate NSFW count and sentiment
.mapValues(avgSentimentAndNSFWCount => {
val totalCommentsAtTime = rddComments.filter(y => y._5 == y._5).count()
val nsfwPercentage = (avgSentimentAndNSFWCount._1.toDouble / totalCommentsAtTime) * 100
val avgSentiment = avgSentimentAndNSFWCount._2 / totalCommentsAtTime
(nsfwPercentage, avgSentiment)
})*/

In [8]:
// PART 3: Join on Temporal Dimension
val finalResult = percentageNSFWPosts.join(avgSentimentWithNSFWPost)
.coalesce(1) //riduzione a singola partizione (per analisi, con singola partizione abbiamo singolo file da analizzare)
.collect()

finalResult: Array[(Int, (Double, (Double, Double)))] = Array((2020,(1.0,(0.0,0.15337666033167532))), (2021,(1.0,(0.0,0.017297035915188295))))
